<a href="https://colab.research.google.com/github/protoi/Speech-Emotion-Analysis/blob/main/3_BandPass_Filtering_Spectrogram_Generation_Image_Manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DESCRIPTION 

Uses the .pickle file generated from the previous code, passes the audio time series through a band pass filter and produces spectrograms.

# LIBRARIES AND IMPORTS

In [ ]:
!pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math as math
import librosa as lib
import librosa.display as lid
import re 
from pydub import AudioSegment
from pydub.silence import split_on_silence
from pydub.playback import play
import os as os
import glob as glob
from pathlib import Path
from os import chdir
from scipy.signal import butter, filtfilt
import gc
from tqdm import tqdm
from google.colab import drive
import shutil
from google.colab import output
import IPython
from IPython.display import Audio
import random as rand
from pylab import Rectangle
import gc 
from scipy import stats
import soundfile as sf
import pyarrow.feather as feather
from scipy.signal import butter
from scipy.signal import butter, filtfilt

# LOADING AND DESERIALIZING THE PICKLE FILE

In [ ]:
# !cp /content/drive/MyDrive/Machine\ Learning/Speech\ Emotion\ Recognition/audio_without_silence.pkl /content/audio_without_silence.pkl
df = None
df = pd.read_pickle("/content/drive/MyDrive/Machine Learning/Speech Emotion Recognition/audio_without_silence.pkl")
df

,labels,source,path,name,silence_removed
0,male_anger,SAVEE,/content/SAVEE/AudioData/DC/a01.wav,DC_a01,"[0.112091064, 0.11282349, 0.088012695, 0.05386..."
1,male_anger,SAVEE,/content/SAVEE/AudioData/DC/a02.wav,DC_a02,"[0.10910034, 0.11364746, 0.10357666, 0.0806579..."
2,male_anger,SAVEE,/content/SAVEE/AudioData/DC/a03.wav,DC_a03,"[0.12271118, 0.123931885, 0.1251831, 0.1264648..."
3,male_anger,SAVEE,/content/SAVEE/AudioData/DC/a04.wav,DC_a04,"[0.12472534, 0.13046265, 0.13531494, 0.1396484..."
4,male_anger,SAVEE,/content/SAVEE/AudioData/DC/a05.wav,DC_a05,"[0.121795654, 0.1227417, 0.11932373, 0.1135559..."
...,...,...,...,...,...
12157,female_disgust,CREMAD,/content/CREMAD/AudioWAV/1091_WSI_DIS_XX.wav,1091_WSI_DIS_XX,"[0.0009117986, 0.0011943923, 0.0013480312, 0.0..."
12158,female_fear,CREMAD,/content/CREMAD/AudioWAV/1091_WSI_FEA_XX.wav,1091_WSI_FEA_XX,"[-0.0065059913, -0.007563202, -0.0075736893, -..."
12159,female_happy,CREMAD,/content/CREMAD/AudioWAV/1091_WSI_HAP_XX.wav,1091_WSI_HAP_XX,"[0.011431874, 0.011645955, 0.011703809, 0.0115..."
12160,female_neutral,CREMAD,/content/CREMAD/AudioWAV/1091_WSI_NEU_XX.wav,1091_WSI_NEU_XX,"[0.008490561, 0.00867801, 0.008760293, 0.00883..."


# SOME STATISTICS

In [ ]:
length_of_audios = np.array([len(i) for i in df['silence_removed'].values])
print(f'mean: {np.mean(length_of_audios)}')
print(f'median: {np.median(length_of_audios)}')
print(f'mode: {stats.mode(length_of_audios).mode[0]}')
print(f'standard deviation: {np.std(length_of_audios)}')

mean: 77560.97845749055
median: 71158.5
mode: 50852
standard deviation: 26487.01623615346


# ADDING & REMOVING SAMPLES TO MAKE EVERY ARRAY THE LENGTH OF THE MEDIAN

In [ ]:
%%time
#makes length of all audio files = median length
def standardize_length(audio_sample, median_length):
  if len(audio_sample) <= 1:
    # return audio_sample #passing a 1 len array to keep the array jagged, numpy will make an array of arrays into aa 2D array if the length of all the arrays are equal
    yield audio_sample #passing a 1 len array to keep the array jagged, numpy will make an array of arrays into aa 2D array if the length of all the arrays are equal

  if len(audio_sample) >= median_length: #more audio samples than we need, just slice it
    # return audio_sample[ : median_length]
    yield audio_sample[ : median_length]
  else:                                   # append it to itself till length = median_length
    reps = int(median_length/len(audio_sample))
    # return np.concatenate((np.tile(audio_sample, reps), audio_sample[: median_length % len(audio_sample)]))
    yield np.concatenate((np.tile(audio_sample, reps), audio_sample[: median_length % len(audio_sample)]))

#calls audio normalized, can make it chain call bandpass filter later if needed
def audio_trimming(numpy_array: np.array, target_len) -> np.array:
  return np.array(list(map(lambda x: standardize_length(x, target_len), numpy_array)), dtype=object)

median_len = math.floor(np.median(length_of_audios))


ragged_row = pd.Series({'labels':None, 'source':None, 'path':None, 'name':None, 'silence_removed':np.array([1])})



#comment the three lines below when testing, this is for production
df = pd.concat([df, ragged_row.to_frame().T], ignore_index=True)
df['audio_normalized'] = audio_trimming(df['silence_removed'].values, median_len)
# df['silence_removed'] = None
# df.drop(columns=['silence_removed', 'path'], inplace=True)


#below codes for testing
# df_temp = df.head(10).copy(deep=True)
# df_temp = pd.concat([df_temp, ragged_row.to_frame().T], ignore_index=True)
# df_temp['audio_normalized'] = audio_trimming(df_temp['silence_removed'].values, median_len)
# df_temp

# dont forget to remove the last row from the dataframe, it was only there to make the array ragged

CPU times: user 135 ms, sys: 4.51 ms, total: 139 ms
Wall time: 140 ms


### PLAYING SOME AUDIOS AS A TEST

In [ ]:
'''import time
randNum = 12

print('WITH SILENCE REMOVED')
sf.write('name.wav', df_temp['silence_removed'][randNum], 44100, subtype='PCM_24')
wn = Audio('name.wav', autoplay = True)
display(wn)

time.sleep(4)
print('LENGTH NORMALIZED')
sf.write('name2.wav', df_temp['audio_normalized'][randNum], 44100, subtype='PCM_24')
wn2 = Audio('name2.wav', autoplay = True)
display(wn2)'''

"import time\nrandNum = 12\n\nprint('WITH SILENCE REMOVED')\nsf.write('name.wav', df_temp['silence_removed'][randNum], 44100, subtype='PCM_24')\nwn = Audio('name.wav', autoplay = True)\ndisplay(wn)\n\ntime.sleep(4)\nprint('LENGTH NORMALIZED')\nsf.write('name2.wav', df_temp['audio_normalized'][randNum], 44100, subtype='PCM_24')\nwn2 = Audio('name2.wav', autoplay = True)\ndisplay(wn2)"

# GENERATING SPECTROGRAMS AND STORING THEM

In [ ]:
#saving the spectrogram
def saveSpectro(data_to_plot, sampling_rate, filename, filesource, filelabel):
  if filename == None:
    return
  fig = plt.figure(num=1, clear=True)
  ax = fig.add_subplot()
  lid.specshow(data_to_plot, sr=sampling_rate, x_axis='time', y_axis='log');
  ax.axis('off')
  fig.savefig(f"spectrograms/{filelabel}-{filesource}-{filename}.png", bbox_inches='tight', pad_inches = 0)
  fig.clear()
  plt.close(fig)






def shortTimeFourier(audio_data):
  try:
    saveSpectro(lib.amplitude_to_db(np.abs(lib.stft(next(next(call_bandpass(next(audio_data['audio_normalized']) , 44100))))),ref=np.max),
              44100,
              audio_data['name'],
              audio_data['source'],
              audio_data['labels'])
  except: 
    print(f'something went wrong with audio file {audio_data["name"]}-{audio_data["source"]}-{audio_data["labels"]}')


def shortTimeFourierWithReturn(audio_data):
  return np.array(list(map(lambda x: lib.amplitude_to_db(np.abs(lib.stft(next(next(call_bandpass(next(x) , 44100))))),ref=np.max) , 
                           audio_data)), dtype=object)
  

def shortTimeFourierGeneratorWithReturn(audio_data):
  return np.array(list(map(lambda x: lib.amplitude_to_db(np.abs(lib.stft(call_bandpass(x , 44100))),ref=np.max) , 
                           audio_data)), dtype=object)
  
  
  # lib.amplitude_to_db(np.abs(lib.stft(next(next(call_bandpass(next(audio_data) , 44100))))),ref=np.max)

#obtaining the spectrogram
# def getSpectroData(row):
#   shortTimeFourier(row, 44100)


#butterworth bandpass filter
def butter_bandpass_filter(lowcutoff, highcutoff, samplingRate, order = 2):
 
  nyquistFreq = 0.5 * samplingRate
  low_normal_cutoff = lowcutoff / nyquistFreq
  high_normal_cutoff = highcutoff / nyquistFreq
  b, a = butter(order, [low_normal_cutoff, high_normal_cutoff], btype = 'band')
  return b, a

#signal filtered
def filter_signal(originalSignal, cutoff_freq, samplingRate, filterType='bandpass', order = 2):

  if filterType.lower() == 'bandpass':
    assert type(cutoff_freq) == list or np.array, 'enter frequencies as a list'
    b,a = butter_bandpass_filter(cutoff_freq[0], cutoff_freq[1], samplingRate, order=order)

  yield filtfilt(b, a, originalSignal)
  # yield filtered_data


#calling the bandpass filter with the raw data
def call_bandpass(data, sampling_rate):

  orderOfFilter = 6
  yield filter_signal(data, [128, 8192], sampling_rate, filterType = 'bandpass', order = orderOfFilter)
  # yield filteredData




In [ ]:
if os.path.exists('/content/spectrograms'):
  shutil.rmtree("/content/spectrograms")
!mkdir spectrograms



copied_df = df.head(3).copy(deep=True)
ragged_row = pd.Series({'labels':None, 'source':None, 'path':None, 'name':None, 'silence_removed':np.array([1]), 'audio_normalized': None})
df = pd.concat([df, ragged_row.to_frame().T], ignore_index=True)


df.apply(shortTimeFourier, axis=1)



something went wrong with audio file 1076_MTI_SAD_XX-CREMAD-female_sadness
something went wrong with audio file None-None-None
something went wrong with audio file None-None-None


'for row in df.itertuples():\n  if row.name == None:\n    break\n  lid.specshow(lib.amplitude_to_db(np.abs(lib.stft(next(next(call_bandpass(next(row.audio_normalized) , 44100))))),ref=np.max),\n               sr=44100,\n               x_axis=\'time\', \n               y_axis=\'log\');\n  fig = plt.figure(num=1, clear=True)\n  ax = fig.add_subplot()\n  ax.axis(\'off\')\n  fig.savefig(f"spectrograms/{row.labels}-{row.source}-{row.name}.png", bbox_inches=\'tight\', pad_inches = 0)\n  fig.clear()\n  plt.close(fig)\n'

In [ ]:
gc.collect()

1712298

In [ ]:
import os as os
print(len(os.listdir('/content/spectrograms')))

12161


imagedatagenerator from keras

# ZIPPING THE IMAGES

In [ ]:
!pwd
!zip -r spectrograms.zip spectrograms/*.png

Streaming output truncated to the last 5000 lines.
  adding: spectrograms/male_anger-CREMAD-1023_ITH_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_ITS_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_IWL_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_IWW_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_MTI_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_TAI_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_TIE_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_TSI_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1023_WSI_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1026_DFA_ANG_XX.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1026_IEO_ANG_HI.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1026_IEO_ANG_LO.png (deflated 1%)
  adding: spectrograms/male_anger-CREMAD-1026_IEO

# MOVING THE ZIPPED FILE TO THE DRIVE

In [ ]:
!cp /content/spectrograms.zip /content/drive/MyDrive/Machine\ Learning/Speech\ Emotion\ Recognition/spectrograms_30_09_2022.zip 
